# RNNs with Law2Vec embeddings

## Importing data, pre-trained embeddings

In [1]:
import copy
from itertools import product
from numpy import isnan
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as data
import torchtext.vocab as vocab
import sys

sys.path.append('../data_pipeline/')
import preprocessing as pre
from training import TrainingModule

In [2]:
train_data, test_data, val_data, TEXT, LABEL = pre.get_data(
    'train_small.csv', 'val_small.csv', 'test_small.csv', 50)

Connected!


/home/ubuntu/bf-repo/preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apply(lambda x: x[0: 1000000])


In [4]:
USE_CUDA = torch.cuda.is_available()

vectors = vocab.Vectors('../embeds/Law2Vec.100d.txt') # Law2Vec available from https://archive.org/details/Law2Vec

TEXT.build_vocab(train_data, vectors=vectors,
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

BATCH_SIZE = 5

train_it, test_it, val_it = data.BucketIterator.splits(
    (train_data, test_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.alj_text),
    sort_within_batch=True,
    device = torch.device('cuda' if USE_CUDA else 'cpu'))

## Checking pretrained vectors have been applied

In [5]:
vectors['medicare']

tensor([ 0.6415, -0.5367, -0.3537, -0.0634, -0.1798,  0.0626, -0.1836, -0.2705,
         0.2504,  0.5061,  0.4746, -0.2351, -0.0465,  0.3184,  0.8974,  0.0470,
        -0.2594,  0.3485, -0.3356,  0.1163,  0.2207,  0.2707,  0.4748,  0.1122,
        -0.1188, -0.0790,  0.4377, -0.4711,  0.1401, -0.0234, -0.2009, -0.2143,
         0.1335, -0.4407,  0.4077, -0.0634,  0.5104,  0.1820, -0.4729, -0.1758,
         0.6194,  0.5708, -0.3034, -0.3658,  0.1609,  0.0753, -0.2024, -0.1472,
         0.0665,  0.1823,  0.3091, -0.0913,  0.2495,  0.0777, -0.1873, -0.5850,
        -0.3243,  0.1540, -0.5094,  0.6227,  0.1163, -0.6202, -0.4416, -0.3509,
        -0.5760, -0.4837, -0.6283,  0.0938,  0.3528, -0.0674, -0.7097, -0.2053,
        -0.6007, -0.1306,  0.0146, -0.0830,  0.5486, -0.2328, -0.3193,  0.1496,
        -0.1635,  0.0755, -0.2594, -0.0317,  0.1249, -0.5599,  0.0722, -0.0369,
         0.3139,  0.0102, -0.3353,  0.1142, -0.1163,  0.1505,  0.0952,  0.0206,
        -0.0733, -0.4851,  0.4995,  0.04

In [6]:
TEXT.vocab.vectors[TEXT.vocab.stoi['medicare']]

tensor([ 0.6415, -0.5367, -0.3537, -0.0634, -0.1798,  0.0626, -0.1836, -0.2705,
         0.2504,  0.5061,  0.4746, -0.2351, -0.0465,  0.3184,  0.8974,  0.0470,
        -0.2594,  0.3485, -0.3356,  0.1163,  0.2207,  0.2707,  0.4748,  0.1122,
        -0.1188, -0.0790,  0.4377, -0.4711,  0.1401, -0.0234, -0.2009, -0.2143,
         0.1335, -0.4407,  0.4077, -0.0634,  0.5104,  0.1820, -0.4729, -0.1758,
         0.6194,  0.5708, -0.3034, -0.3658,  0.1609,  0.0753, -0.2024, -0.1472,
         0.0665,  0.1823,  0.3091, -0.0913,  0.2495,  0.0777, -0.1873, -0.5850,
        -0.3243,  0.1540, -0.5094,  0.6227,  0.1163, -0.6202, -0.4416, -0.3509,
        -0.5760, -0.4837, -0.6283,  0.0938,  0.3528, -0.0674, -0.7097, -0.2053,
        -0.6007, -0.1306,  0.0146, -0.0830,  0.5486, -0.2328, -0.3193,  0.1496,
        -0.1635,  0.0755, -0.2594, -0.0317,  0.1249, -0.5599,  0.0722, -0.0369,
         0.3139,  0.0102, -0.3353,  0.1142, -0.1163,  0.1505,  0.0952,  0.0206,
        -0.0733, -0.4851,  0.4995,  0.04

## RNN Model

In [18]:
class RNNPtEmbeds(nn.Module):
    
    def __init__(self, rnn_type, input_size, embedding_size,
                 hidden_size, output_size, num_layers,
                 dropout, bidirectional, padding_idx):
    
        super().__init__()
        self.embedding = nn.Embedding\
                           .from_pretrained(TEXT.vocab.vectors)

        self.rnn = getattr(nn, rnn_type.upper())\
                          (embedding_size, hidden_size, num_layers,
                           dropout=(dropout if num_layers > 1 else 0),
                           bidirectional=bidirectional)
        
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout)
        
        linear_inp = (hidden_size * 2 if bidirectional else hidden_size)
        self.linear = nn.Linear(linear_inp, output_size)
             
    def forward(self, input):
        embed = self.embedding(input)
        rnn_out, hidden = self.rnn(embed)
        rnn_out = rnn_out[-1]
        rnn_out = self.leakyrelu(rnn_out)
        rnn_out = self.dropout(rnn_out)
        linear_out = self.linear(rnn_out)
        return linear_out

## Training models

In [20]:
# Store training results
df = pd.DataFrame(columns=['architecture', 'model_type', 'embeddings',
                           'hidden', 'num_layers', 'dropouts',
                           'bidirectional', 'learning_rate', 'epochs',
                           'dev_acc', 'dev_prec', 'dev_recall',
                           'metric'])

# Model architecture parameters
RNN_TYPES = ['RNN', 'LSTM']
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_SIZE = TEXT.vocab.vectors.size(1)
HIDDEN_SIZES = [1/3, 2/3]
OUTPUT_SIZE = 1
NUM_LAYERS = [1, 2]
DROPOUTS = [0.5, 0.75]
BIDIRECTIONALS = [False, True]
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Model training hyperparameters
LEARNING_RATE = [0.01, 0.0001]
train_len = 0
train_pos = 0
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()
EPOCHS = 10

# Iterator over various model parameters
param_iter = product (RNN_TYPES, HIDDEN_SIZES, NUM_LAYERS,
                      DROPOUTS, BIDIRECTIONALS, LEARNING_RATE)

# Magic loop
best_acc = (None, None)
best_rec = (None, None)
best_prec = (None, None)
for i, (rnn_type, hidden_size, num_layers, dropout, bidirectional,\
    lr) in enumerate(param_iter):
    print(f'Architecture #{i}\n' + '-' * 20)
    hidden_dim = int(hidden_size * EMBEDDING_SIZE)
    model = RNNPtEmbeds(rnn_type, INPUT_DIM, EMBEDDING_SIZE,
                        hidden_dim, OUTPUT_SIZE, num_layers,
                        dropout, bidirectional, PADDING_IDX)
    
    tm = TrainingModule(model, lr, POS_WEIGHT, USE_CUDA, EPOCHS)
    
    best_models = tm.train_model(train_it, val_it)
    
    for metric, best_model in best_models.items():
        row = [i, rnn_type, 'Law2Vec', hidden_size, num_layers, dropout,
               bidirectional, lr, EPOCHS, best_model.accuracy,
               best_model.precision, best_model.recall, metric]
        df.loc[len(df)] = row
        if best_acc[0] is None or best_model.accuracy > best_acc[1]:
            best_acc = (copy.deepcopy(best_model.model), best_model.accuracy)
        if best_rec[0] is None or best_model.recall > best_rec[1]:
            best_rec = (copy.deepcopy(best_model.model), best_model.recall)
        if best_prec[0] is None or isnan(best_prec[1]) or\
           best_model.precision > best_prec[1]:
            best_prec = (copy.deepcopy(best_model.model), best_model.precision)
    
    print('-' * 20 + '\n')


Architecture #0
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8611
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8672
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8560
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8528
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8442
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8404
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8241
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8229
Epoch 8: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8166
Epoch 9: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8022
Epoch 10: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7

## Save results of model training

In [ ]:
SAVE_PREFIX = '../results/RNNLaw2Vec_'
df.to_csv(f'{SAVE_PREFIX}models.csv')
torch.save(best_acc[0], f'{SAVE_PREFIX}best_acc.pt')
torch.save(best_rec[0], f'{SAVE_PREFIX}best_rec.pt')
torch.save(best_prec[0], f'{SAVE_PREFIX}best_prec.pt')

## Confirming embeddings have not been trained

In [22]:
medicare_tensor = torch.LongTensor([TEXT.vocab.stoi['medicare']])
if USE_CUDA:
    medicare_tensor = medicare_tensor.cuda()
model.embedding(medicare_tensor)

tensor([[ 0.6415, -0.5367, -0.3537, -0.0634, -0.1798,  0.0626, -0.1836, -0.2705,
          0.2504,  0.5061,  0.4746, -0.2351, -0.0465,  0.3184,  0.8974,  0.0470,
         -0.2594,  0.3485, -0.3356,  0.1163,  0.2207,  0.2707,  0.4748,  0.1122,
         -0.1188, -0.0790,  0.4377, -0.4711,  0.1401, -0.0234, -0.2009, -0.2143,
          0.1335, -0.4407,  0.4077, -0.0634,  0.5104,  0.1820, -0.4729, -0.1758,
          0.6194,  0.5708, -0.3034, -0.3658,  0.1609,  0.0753, -0.2024, -0.1472,
          0.0665,  0.1823,  0.3091, -0.0913,  0.2495,  0.0777, -0.1873, -0.5850,
         -0.3243,  0.1540, -0.5094,  0.6227,  0.1163, -0.6202, -0.4416, -0.3509,
         -0.5760, -0.4837, -0.6283,  0.0938,  0.3528, -0.0674, -0.7097, -0.2053,
         -0.6007, -0.1306,  0.0146, -0.0830,  0.5486, -0.2328, -0.3193,  0.1496,
         -0.1635,  0.0755, -0.2594, -0.0317,  0.1249, -0.5599,  0.0722, -0.0369,
          0.3139,  0.0102, -0.3353,  0.1142, -0.1163,  0.1505,  0.0952,  0.0206,
         -0.0733, -0.4851,  